# Exercise 2 - Trees
Vi ska använda tre modeller för att avgöra om en person överlever Titanic eller inte
Uppgift:
1. Använd din preparerade datafil från föregånede Exercise 1, alternativt använd den förpreparerade titanic.csv-filen.
2. Välj ut och skapa dina egna features
3. Dela upp ditt dataset i träning och test

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
df = pd.read_csv('titanic.csv')

In [9]:
# En schysst funktion för att göra någonting av kabin-nummer. Henrik delar upp cabin i CabinSection och CabinNumber 
# (baserat på bokstaven framför, och siffran efteråt).
def dictionary_function(df, col):
    my_value_list = sorted(list(set([item[0] for item in list(set(list(df[col].values)))])))
    my_ranking_list = list(range(len(my_value_list)))
    my_dictionary = {}
    for x,y in zip(my_value_list, my_ranking_list):
        my_dictionary[x] = y
    df.replace({col: my_dictionary}, inplace=True)
    return df

# Dumma kolumner tas bort.
dumb_cols = ['Unnamed: 0', 'Unnamed: 0.1']
for col in dumb_cols:
    if col in df.columns:
        df.drop(columns={col}, inplace=True)

# Kolumnen Survived har en jäkla massa null-värden. Vi får ta bort dessa. Inte kul, men raderna ger oss ingenting.
df = df[df['Survived'].notna()]

# Kolumnen Survived är strängar just nu. Vi gör om det till ints.
df['Survived'].replace({'Yes': 1, 'No':0}, inplace=True)
df['Survived'] = df['Survived'].astype(int)

# Vi fyller på avsaknade värden i kolumnen Age med genomsnittlig ålder. Vi 
df['Age'].fillna(round(df['Age'].mean()), inplace=True)
df['Age'] = df['Age'].apply(lambda x: df['Age'].mean() if x > 120 else x)
df['Age'] = df['Age'].astype(int)

# Vi fyller på avsaknade värden i kolumnen Embarked med Unknown.
df.Embarked.fillna('U', inplace=True)

# Vi fyller på avsaknade värden i kolumnen Cabin med U0.
df.Cabin.fillna('U0', inplace=True)

# Vi gör om kön till 1 och 0
df['Sex'] = df['Sex'].replace({'male':1,'female':0})

# Vi splittar vår kolumn Cabin i section och hyttnymmer.
# Sen använder vi vår schyssta funktion för att extrahera kabinsektion och hyttnummer.
df[['CabinSection', 'CabinNr', 'dummy']] = df["Cabin"].str.split("(\d+)", n=1, expand=True)
df.CabinSection = df.CabinSection.apply(lambda x: x[0])
df.CabinNr.fillna(0, inplace=True)
df.CabinNr = df.CabinNr.astype(int)
for col in ['CabinSection', 'Embarked']:
    df = dictionary_function(df, col)
    df[col].astype(int)

# Vi droppar alla konstiga kolumner som dykt upp under processen och som inte påverkar överlevnadschanserna.
df.drop(columns={'dummy', 'Cabin', 'Ticket', 'Name', 'PassengerId'}, inplace=True)

# Vi skalar/normaliserar vårt dataset
target_col = ['Survived']
feature_cols = [col for col in df.columns if col not in target_col]

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler_fit = scaler.fit(df[feature_cols])
df[feature_cols] = pd.DataFrame(scaler.transform(df[feature_cols]), index=df.index, columns=feature_cols)


# Vi definierar vårt target och våra features.
y = df[target_col]
X = df[feature_cols]

# Vi delar upp vårt dataset i train och test.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [10]:
X_train

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,CabinSection,CabinNr
445,0.0,1.0,0.0500,0.000,0.333333,0.159777,0.666667,0.000,0.229730
650,1.0,1.0,0.3750,0.000,0.000000,0.015412,0.666667,1.000,0.000000
172,1.0,0.0,0.0125,0.125,0.166667,0.021731,0.666667,1.000,0.000000
450,0.5,1.0,0.4500,0.125,0.333333,0.054164,0.666667,1.000,0.000000
314,0.5,1.0,0.5375,0.125,0.166667,0.051237,0.666667,1.000,0.000000
...,...,...,...,...,...,...,...,...,...
106,1.0,0.0,0.2625,0.000,0.000000,0.014932,0.666667,1.000,0.000000
270,0.0,1.0,0.3750,0.000,0.000000,0.060508,0.666667,1.000,0.000000
860,1.0,1.0,0.5125,0.250,0.000000,0.027538,0.666667,1.000,0.000000
435,0.0,0.0,0.1750,0.125,0.333333,0.234224,0.666667,0.125,0.648649


## Decision Tree
1. Använd ett decision tree, en random forest och XGBoost för att skapa predictions
2. Plotta trädet för att se hur det ser ut. Går det att förstå något?

In [6]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
model = DecisionTreeClassifier() # optimize! Can we do it better?

## Random Forest
1. Använd en random forest för att skapa predictions.
2. Är denna bättre än vårt Decision Tree?
3. En RandomForest-modell består av flera DecisionTrees. Dessa går att få fram genom model_forest.estimators_. Plotta de 5 första träden i vår RandomForest för att se hur det ser ut. Går det att förstå något?

In [9]:
from sklearn.ensemble import RandomForestClassifier
model_forest = RandomForestClassifier() # optimize! Can we do it better?

/var/folders/t3/c9yl4b8s3fd5vzr9v1gp94ym0000gp/T/ipykernel_26632/1013424532.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model_forest.fit(X_train, y_train)


## XGBoost
1. Använd en XGBoostClassifier för att skapa predictions.
2. Är denna bättre än vårt Decision Tree och RandomForest?
3. Använd from xgboost import plot_tree för att plotta vår XGBoostClassifier för att se hur det ser ut. Går det att förstå något?

In [12]:
import xgboost as xgb
from xgboost import plot_tree
model_XGB = xgb.XGBClassifier() # optimize! Can we do it better?